1. 사람 + 총 + 칼 인식    
2. Arcface를 모든 사람에 대해 적용(Me, NotME)  
3. NotMe이면서 사람과 총 혹은 칼의 bounding box가 겹치는 경우에는 위험인으로 분류 -> 한 번 위험인으로 분류된 사람의 ID는 dangerous_ids로 관리됨 -> dangerous_ids 중 하나에 해당하는 사람은 위에 "Dangerous person"이라고 뜸
4. pose estimation을 dangerous person에 대해 적용(왼팔들기, 오른팔들기, 양팔들기)  
5. 출력화면 사이즈를 키움  
6. 버그수정: 초기에 무기와 교차하여 dangerous_ids에 들어갔더라도, ArcFace로 내 얼굴임이 확인되면(=나 자신이 무기를 소지한 상황) 그 사람을 위험 인물에서 제외  
7. Kobukki robot이 dangerous_ids 내에 있는 사람을 쫓아가도록 하면 될 듯  

+ Yolo segmentation까지 추가
+ 현재 YOLO를 사용하고 있으므로, YOLOv8-seg 모델을 사용하는 것이 가장 간단함.

총이나 칼을 들고 있는 손의 정확한 위치를 파악해서 정밀하게 무기를 감지할 수 있음
옷 색깔이나 패턴을 인식해서 사람 구별 정확도 높아짐
제스처 구분 정확도 높아짐 
YOLOv8-seg 모델 사용


- v2: seg 모델 학습되어 있는 거 가져와서 테스트 (이전에 학습시켜두었던 yolo 모델은 가져오지 않음). seg + arcface
- v3: seg + arcface + deepsort (사람)
- v4: seg(사람) + arcface + deepsort + detection(사람,총,칼)
detection은 이전에 학습시켰던 epoch180.pt 사용
detection먼저 하고 사람인 경우에만 seg


In [1]:
#1. 얼굴 학습시키는 부분
import cv2
import numpy as np
import os
from insightface.app import FaceAnalysis

def initialize_arcface():
    app = FaceAnalysis(name="buffalo_l")  # ArcFace 모델 (buffalo_l은 기본 권장)
    app.prepare(ctx_id=-1, det_size=(640, 640))  # GPU: ctx_id=0, CPU: -1
    return app

def get_face_embedding(app, image_bgr):
    # ArcFace의 app.get()은 BGR 형식으로 이미지를 받기도 합니다.
    # 만약 RGB가 필요하면 cvtColor로 변환하세요.
    faces = app.get(image_bgr)
    if len(faces) > 0:
        return faces[0].embedding  # 첫 번째 얼굴의 임베딩
    else:
        return None

def generate_average_embedding(app, folder_path):
    embeddings = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"이미지 로드 실패: {img_path}")
                continue
            
            embedding = get_face_embedding(app, image)
            if embedding is not None:
                embeddings.append(embedding)
            else:
                print(f"얼굴 검출 실패: {img_path}")
    
    if len(embeddings) == 0:
        raise ValueError("임베딩을 하나도 생성하지 못했습니다.")
    
    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding

if __name__ == "__main__":
    app = initialize_arcface()
    # 내 얼굴 사진 폴더
    my_face_folder = "C:/myface"  
    my_face_embedding = generate_average_embedding(app, my_face_folder)
    np.save("my_face_embedding.npy", my_face_embedding)  # 필요 시 저장
    print("내 얼굴 평균 임베딩 생성 완료.")

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


내 얼굴 평균 임베딩 생성 완료.


In [2]:
import cv2
import numpy as np
import time
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity
import mediapipe as mp

############################
# 1) 모델 및 함수 초기화
############################
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# YOLO Detection 모델 로드
detection_model = YOLO("C:/epoch180.pt").to(device)

# YOLO Segmentation 모델 로드
segmentation_model = YOLO("yolov8n-seg.pt").to(device)

# DeepSORT 초기화
tracker = DeepSort(max_age=30, n_init=3, embedder='mobilenet', half=True, embedder_gpu=True)

# ArcFace 초기화
arc_app = FaceAnalysis(name="buffalo_l")
arc_app.prepare(ctx_id=-1, det_size=(640, 640))  # ctx_id=-1: CPU, ctx_id=0: GPU
my_face_embedding = np.load("my_face_embedding.npy")  # 내 얼굴의 평균 임베딩

# MediaPipe Pose Estimation 초기화
mp_pose = mp.solutions.pose
pose_model = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def get_face_embedding(face_image, arc_app):
    faces = arc_app.get(face_image)
    if len(faces) > 0:
        return faces[0].embedding
    return None

def is_my_face(face_embedding, my_face_embedding, threshold=0.4):
    if face_embedding is None:
        return False, 0.0
    similarity = cosine_similarity([face_embedding], [my_face_embedding])[0][0]
    return similarity > threshold, similarity

############################
# 2) 실시간 웹캡 캡처
############################
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

window_name = "YOLOv8 + DeepSORT + Segmentation + ArcFace + Pose"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 960, 720)

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    # YOLOv8 Detection 실행
    detection_results = detection_model(frame)

    # Segmentation 마스크를 위한 빈 프레임
    mask_overlay = np.zeros_like(frame, dtype=np.uint8)

    # DeepSORT 입력을 위한 검출된 객체 리스트
    detections = []

    for box in detection_results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
        class_id = int(box.cls)  # 클래스 ID
        conf = float(box.conf)  # 신뢰도 점수
        label = f"{detection_model.names[class_id]}: {conf:.2f}"

        # 바운딩 박스 그리기
        color = (0, 255, 0) if class_id == 0 else (0, 0, 255)  # 사람: 초록색, 그 외: 빨간색
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # DeepSORT 입력 추가 (사람만)
        if class_id == 0:
            detections.append(((x1, y1, x2 - x1, y2 - y1), conf, 0))  # DeepSORT에 추가

    # DeepSORT 추적
    tracks = tracker.update_tracks(detections, frame=frame)

    # Segmentation, ArcFace 및 Pose Estimation 적용
    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue

        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())

        # Tracking ID 표시
        cv2.putText(frame, f"ID:{track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        # Segmentation 적용 (사람만)
        person_crop = frame[y1:y2, x1:x2]
        if person_crop.size == 0:
            continue

        # Segmentation 실행
        segmentation_results = segmentation_model(person_crop)
        if segmentation_results[0].masks is not None:
            for mask in segmentation_results[0].masks.data:
                mask = mask.cpu().numpy()
                mask_resized = cv2.resize(mask, (x2 - x1, y2 - y1))

                # 마스크 오버레이
                mask_binary = (mask_resized > 0.5).astype(np.uint8)
                color_mask = np.zeros_like(person_crop, dtype=np.uint8)
                color_mask[:, :, 1] = mask_binary * 255  # 초록색 채널
                person_crop = cv2.addWeighted(person_crop, 1, color_mask, 0.5, 0)
                frame[y1:y2, x1:x2] = person_crop

        # ArcFace 얼굴 인식
        face_embedding = get_face_embedding(person_crop, arc_app)
        is_me, similarity = is_my_face(face_embedding, my_face_embedding, threshold=0.4)

        if is_me:
            arcface_text = f"Me (sim={similarity:.2f})"
            cv2.putText(frame, arcface_text, (x1, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            arcface_text = f"Not Me (sim={similarity:.2f})"
            cv2.putText(frame, arcface_text, (x1, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Pose Estimation (왼팔, 오른팔 감지)
        crop_rgb = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
        pose_result = pose_model.process(crop_rgb)

        if pose_result.pose_landmarks:
            landmarks = pose_result.pose_landmarks.landmark
            left_wrist_y = landmarks[15].y  # 왼손
            right_wrist_y = landmarks[16].y  # 오른손
            left_shoulder_y = landmarks[11].y  # 왼쪽 어깨
            right_shoulder_y = landmarks[12].y  # 오른쪽 어깨

            action_text = ""
            if left_wrist_y < left_shoulder_y - 0.05:
                action_text = "Left arm up"
            if right_wrist_y < right_shoulder_y - 0.05:
                action_text = "Right arm up"

            if action_text:
                cv2.putText(frame, action_text, (x1, y1 + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # 화면에 표시
    cv2.imshow(window_name, frame)

    # ESC 또는 'q' 키로 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\user/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 640x416 (no detections), 131.1ms
Speed: 10.3ms preprocess, 131.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 416)

0: 608x640 (no detections), 159.6ms
Speed: 21.4ms preprocess, 159.6ms inference, 8.0ms postprocess per image at shape (1, 3, 608, 640)


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


...

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 640x416 1 person, 94.3ms
Speed: 7.7ms preprocess, 94.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 416)

0: 608x640 (no detections), 129.6ms
Speed: 12.6ms preprocess, 129.6ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)

0: 480x640 2 personnnns, 82.6ms
Speed: 20.1ms preprocess, 82.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 512x640 1 person, 104.0ms
Speed: 7.6ms preprocess, 104.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4



0: 640x416 (no detections), 98.9ms
Speed: 31.6ms preprocess, 98.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 416)

0: 608x640 1 cup, 158.3ms
Speed: 24.0ms preprocess, 158.3ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 640)

0: 480x640 1 personnnn, 78.9ms
Speed: 18.3ms preprocess, 78.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 512x640 1 person, 102.1ms
Speed: 2.0ms preprocess, 102.1ms inference, 8.1ms postprocess per image at shape (1, 3, 512, 640)


ValueError: could not broadcast input array from shape (416,530) into shape (416,525)

In [ ]:
import torch

print("CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())